<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c5cde4f7aa0848d43b7e61cce6822408d7e8a5464ef7add40ef372f59068488b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-09 11:24:11
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.42 C
-------------------
Today PnL: -50.91 K (-0.36%)
Current PnL: -26.93 L (-17.17%)
CY Booked + Current PnL: -12.64 L (-8.06%)
-------------------
Total profit:  1.81 L
Total loss:  -28.73 L
-------------------
Total Booked + Current PnL: 14.34 L (11.05%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.05%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 92.38 L (64.99%)
Deployed:  1.30 C
Current:  1.42 C
CAGR/XIRR %: 4.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.60,14.53,6.00,21.40,18239.0,38563.0,303989.0,6.33,60.0,X-LC,5.70,14.0,2.11,2.14,21.78,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,0.17,-21.27,27.14,0.09,21533.0,-21437.0,79340.0,84.20,36.0,M-SC,3.31,253.0,-1.00,0.56,4.24,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.89,-12.49,14.29,0.02,21829.0,-21803.0,152756.0,132.70,60.0,M-SC,8.11,240.0,-1.00,1.07,32.87,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.48,4.07,14.56,19.23,22799.0,6124.0,156584.0,22.81,71.0,L-LC,7.11,182.0,0.27,1.10,26.21,XY25,ATH,MINING
49,MASFIN,398.61,-1.87,-3.48,26.44,22.05,25006.0,-3405.0,94575.0,-16.89,47.0,H-SC,6.52,168.0,-0.14,0.67,37.47,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,2.34,-17.25,51.67,25.51,79572.0,-32095.0,154000.0,-5.41,46.0,H-LC,1.72,50.0,-0.4,1.08,13.49,AR,ATH,ELECTRICAL
68,STARHEALTH,761.00,2.19,-11.48,66.36,47.26,165139.0,-32270.0,248853.0,20.72,48.0,H-SC,7.61,174.0,-0.2,1.75,33.33,XY24,NTT,INSURANCE
69,SURYODAY,216.00,2.13,-15.65,45.87,23.03,69283.0,-28029.0,151042.0,64.47,64.0,H-SC,4.18,165.0,-0.4,1.06,49.86,XR,NTT,BANKS
44,JCHAC,2282.00,1.36,-33.96,51.40,-0.01,43386.0,-43396.0,84409.0,17182.61,52.0,M-SC,13.73,235.0,-1.0,0.59,8.03,OX40N,BTT,AC
32,HCLTECH,1922.01,1.32,8.27,15.20,24.73,39815.0,20015.0,261939.0,13.18,57.0,X-LC,6.27,13.0,0.5,1.84,24.77,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,-4.12,-32.75,107.60,39.61,74169.0,-33570.0,68930.0,-54.49,50.0,H-SC,30.24,136.0,-0.45,0.48,11.57,AR,ATH,MISC
76,TRIDENT,48.00,-3.91,-30.25,84.26,28.51,54216.0,-27910.0,64344.0,-17.44,37.0,M-SC,7.45,226.0,-0.51,0.45,10.10,XR,NTT,TEXTILES
4,ALKYLAMINE,4546.37,-3.70,-34.07,193.69,93.63,128928.0,-34399.0,66564.0,-29.39,40.0,H-SC,16.67,143.0,-0.27,0.47,0.63,SR,ATH,CHEMICALS
61,ROUTE,2227.21,-2.91,-48.87,230.64,69.05,156914.0,-65032.0,68034.0,-58.67,42.0,H-SC,18.88,140.0,-0.41,0.48,4.92,SR,ATH,IT
28,GREENPANEL,537.00,-2.28,-40.99,142.58,43.14,154028.0,-75049.0,108029.0,142.25,32.0,M-SC,12.80,231.0,-0.49,0.76,1.62,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.54,0.0,20.74,20.74,42137.0,0.0,203170.0,31.15,57.0,M-MC,5.38,189.0,0.0,1.43,10.62,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.35,-2.09,112.36,107.91,162418.0,-3093.0,144551.0,-20.15,52.0,M-SC,10.11,220.0,-0.02,1.02,7.78,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.89,-12.49,14.29,0.02,21829.0,-21803.0,152756.0,132.70,60.0,M-SC,8.11,240.0,-1.00,1.07,32.87,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,0.68,-17.05,59.17,32.03,64705.0,-22481.0,109354.0,-10.16,67.0,H-SC,6.87,157.0,-0.35,0.77,22.49,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.17,-21.27,27.14,0.09,21533.0,-21437.0,79340.0,84.20,36.0,M-SC,3.31,253.0,-1.00,0.56,4.24,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.23,-21.52,44.59,13.47,94364.0,-58041.0,211626.0,-67.73,51.0,H-SC,4.55,159.0,-0.62,1.49,7.61,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.48,4.07,14.56,19.23,22799.0,6124.0,156584.0,22.81,71.0,L-LC,7.11,182.0,0.27,1.10,26.21,XY25,ATH,MINING
84,WIPRO,420.00,0.48,8.06,59.42,72.27,108797.0,13650.0,183098.0,-7.44,51.0,M-LC,6.20,101.0,0.13,1.29,14.73,XR,NTT,IT
37,IEX,219.00,-0.13,1.73,46.10,48.63,93774.0,3460.0,203414.0,-32.21,62.0,H-SC,13.39,137.0,0.04,1.43,13.55,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.48,8.06,59.42,72.27,108797.0,13650.0,183098.0,-7.44,51.0,M-LC,6.20,101.0,0.13,1.29,14.73,XR,NTT,IT
37,IEX,219.00,-0.13,1.73,46.10,48.63,93774.0,3460.0,203414.0,-32.21,62.0,H-SC,13.39,137.0,0.04,1.43,13.55,XR,NTT,MISC
25,FINCABLES,1641.55,-0.35,-2.09,112.36,107.91,162418.0,-3093.0,144551.0,-20.15,52.0,M-SC,10.11,220.0,-0.02,1.02,7.78,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.95,-8.70,126.42,106.72,142354.0,-10732.0,112604.0,-54.82,32.0,H-SC,7.44,138.0,-0.08,0.79,12.64,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.51,-4.78,42.44,35.63,84690.0,-10022.0,199552.0,-18.07,39.0,H-MC,3.15,119.0,-0.12,1.40,11.09,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.73,-17.06,33.57,10.78,66911.0,-41000.0,199318.0,-50.55,12.0,X-LC,6.29,1.0,-0.61,1.40,0.00,X40,NTT,FMCG
7,AWL,485.00,-0.72,-26.69,114.41,57.19,273546.0,-87034.0,239093.0,-65.29,26.0,X-MC,9.21,58.0,-0.32,1.68,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,-0.60,-12.54,49.34,30.62,102297.0,-29724.0,207330.0,-37.83,27.0,X-MC,16.40,57.0,-0.29,1.46,0.00,X40,ATH,APPARELS
59,RELIANCE,1952.00,0.05,-5.55,32.66,25.30,61992.0,-11152.0,189811.0,-8.43,32.0,X-LC,8.07,26.0,-0.18,1.34,27.14,XY25,BTT,REFINERIES
11,BATAINDIA,2096.00,0.16,-40.10,125.72,35.20,96897.0,-51596.0,77074.0,1.08,33.0,X-SC,17.32,93.0,-0.53,0.54,0.16,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.33,-1.14,22.16,20.77,51296.0,-2666.0,231478.0,-9.72,33.0,X-MC,0.95,70.0,-0.05,1.63,5.70,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-0.58,-6.12,32.60,24.48,81823.0,-16374.0,250991.0,-7.66,42.0,X-LC,1.72,37.0,-0.20,1.77,45.48,XY24,BTT,FINANCE
65,SIEMENS,4671.50,2.34,-17.25,51.67,25.51,79572.0,-32095.0,154000.0,-5.41,46.0,H-LC,1.72,50.0,-0.40,1.08,13.49,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.77,2.34,40.41,43.70,102590.0,5805.0,253873.0,-6.13,67.0,X-MC,2.25,73.0,0.06,1.79,17.74,XY24,BTT,FMCG
81,VBL,671.64,-0.61,0.57,34.91,35.67,110884.0,1786.0,317628.0,-11.39,60.0,X-LC,2.26,5.0,0.02,2.23,14.50,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.32,-27.80,127.37,64.15,218809.0,-66161.0,171790.0,-57.62,35.0,X-SC,6.25,84.0,-0.30,1.21,0.00,XY24,BTT,CEMENT
7,AWL,485.00,-0.72,-26.69,114.41,57.19,273546.0,-87034.0,239093.0,-65.29,26.0,X-MC,9.21,58.0,-0.32,1.68,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,-0.60,-12.54,49.34,30.62,102297.0,-29724.0,207330.0,-37.83,27.0,X-MC,16.40,57.0,-0.29,1.46,0.00,X40,ATH,APPARELS
43,ITC,452.00,-0.73,-17.06,33.57,10.78,66911.0,-41000.0,199318.0,-50.55,12.0,X-LC,6.29,1.0,-0.61,1.40,0.00,X40,NTT,FMCG
11,BATAINDIA,2096.00,0.16,-40.10,125.72,35.20,96897.0,-51596.0,77074.0,1.08,33.0,X-SC,17.32,93.0,-0.53,0.54,0.16,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.03,-28.02,98.44,42.84,46064.0,-18212.0,46794.0,-52.62,50.0,X-SC,27.04,83.0,-0.40,0.33,5.87,XY24,NTT,MISC
58,RELAXO,1176.00,-0.46,-48.45,195.44,52.31,146254.0,-70327.0,74833.0,-45.12,39.0,X-SC,7.83,92.0,-0.48,0.53,0.17,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.16,-40.10,125.72,35.20,96897.0,-51596.0,77074.0,1.08,33.0,X-SC,17.32,93.0,-0.53,0.54,0.16,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-0.12,0.16,23.87,24.08,30353.0,208.0,127158.0,-7.89,51.0,X-SC,6.22,91.0,0.01,0.89,19.81,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.09,-17.24,73.17,43.32,98633.0,-28072.0,134800.0,-28.92,46.0,X-SC,7.84,90.0,-0.28,0.95,3.50,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.73,-17.06,33.57,10.78,66911.0,-41000.0,199318.0,-50.55,12.0,X-LC,6.29,1.0,-0.61,1.40,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.75,-26.88,68.19,22.98,112633.0,-60723.0,165175.0,-24.30,44.0,X-LC,4.11,2.0,-0.54,1.16,3.89,XY24,NTT,AUTO
73,TCS,4389.97,0.32,-11.63,36.59,20.71,122304.0,-43981.0,334256.0,-23.95,48.0,X-LC,2.49,3.0,-0.36,2.35,11.69,X40,ATH,IT
81,VBL,671.64,-0.61,0.57,34.91,35.67,110884.0,1786.0,317628.0,-11.39,60.0,X-LC,2.26,5.0,0.02,2.23,14.50,X40N,ATH,FMCG
41,INFY,1972.00,0.79,10.69,21.28,34.24,73701.0,33450.0,346338.0,-13.85,53.0,X-LC,7.71,6.0,0.45,2.44,19.86,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.38,-34.53,102.98,32.89,52054.0,-26658.0,50548.0,-724.57,59.0,L-MC,7.74,259.0,-0.51,0.36,39.20,XR,NTT,BANKS
13,BSOFT,831.70,-0.37,-22.03,93.40,50.79,102425.0,-30990.0,109663.0,0.12,51.0,H-SC,10.71,171.0,-0.30,0.77,27.68,XR,ATH,IT
69,SURYODAY,216.00,2.13,-15.65,45.87,23.03,69283.0,-28029.0,151042.0,64.47,64.0,H-SC,4.18,165.0,-0.40,1.06,49.86,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.89,-12.49,14.29,0.02,21829.0,-21803.0,152756.0,132.70,60.0,M-SC,8.11,240.0,-1.00,1.07,32.87,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.48,4.07,14.56,19.23,22799.0,6124.0,156584.0,22.81,71.0,L-LC,7.11,182.0,0.27,1.10,26.21,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.01,-28.86,58.70,12.90,91242.0,-63058.0,155438.0,129.57,64.0,H-SC,10.21,173.0,-0.69,1.09,29.25,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,0.68,-17.05,59.17,32.03,64705.0,-22481.0,109354.0,-10.16,67.0,H-SC,6.87,157.0,-0.35,0.77,22.49,OX40N,NTT,IT
17,COALINDIA,484.83,-0.48,4.07,14.56,19.23,22799.0,6124.0,156584.0,22.81,71.0,L-LC,7.11,182.0,0.27,1.10,26.21,XY25,ATH,MINING
69,SURYODAY,216.00,2.13,-15.65,45.87,23.03,69283.0,-28029.0,151042.0,64.47,64.0,H-SC,4.18,165.0,-0.40,1.06,49.86,XR,NTT,BANKS
36,ICICIPRULI,790.00,1.14,14.04,14.63,30.73,32868.0,27658.0,224663.0,-12.66,73.0,X-MC,6.64,79.0,0.84,1.58,28.60,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.33
1,25,44.34
2,50,76.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.81
MC    30.73
LC    26.47
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.37
X40      23.89
X40N     12.80
XY25     10.69
XR        9.07
AR        8.53
OX40N     7.71
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.34
X-LC    23.00
H-SC    22.91
M-SC    11.31
X-SC     7.81
H-MC     4.60
M-MC     1.43
M-LC     1.29
L-LC     1.10
H-LC     1.08
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.05,-8.59,44.22
IT,12.88,-16.71,75.58
FINANCE,11.07,-13.41,61.01
MISC,7.12,-28.22,80.67
ELECTRICAL,5.94,-11.17,51.89
PAINTS,5.76,-12.89,29.65
INSURANCE,4.74,-0.64,35.08
PHARMA,3.92,-5.53,38.75
BANKS,2.78,-35.26,117.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3225581.0,21
AR,1356760.0,10
XR,1203002.0,12
X40,1125461.0,15
X40N,923544.0,9
OX40N,695588.0,10
XY25,422599.0,7
SR,285842.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3537533.0,24
X-MC,1575924.0,16
M-SC,1451512.0,15
X-LC,954924.0,13
X-SC,808700.0,8
H-MC,424538.0,3
L-SC,179887.0,2
M-LC,108797.0,1
H-LC,79572.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1250390.0      6
           AR         925118.0      5
M-SC       XY24       839937.0      6
H-SC       XR         770340.0      7
X-MC       X40        510073.0      7
           XY24       443198.0      3
X-LC       X40        418367.0      6
X-MC       X40N       399630.0      4
X-SC       X40N       316453.0      3
H-SC       OX40N      305843.0      4
M-SC       OX40N      297523.0      5
X-SC       XY24       295226.0      3
H-SC       SR         285842.0      2
X-MC       XY25       223023.0      2
H-MC       AR         222164.0      2
X-LC       X40N       207461.0      2
H-MC       XY24       202374.0      1
X-SC       X40        197021.0      2
X-LC       XY24       194456.0      2
M-SC       XR         184146.0      2
X-LC       XY25       134640.0      3
M-SC       AR         129906.0      2
M-LC       XR         108797.0      1
L-SC       OX40N       92222.0      1
           XR          87665.0      1
H-LC       AR          79572.0      1
L-MC       XR          52054.0      1
M-MC       XY25        42137.0      1
L-LC       XY25        22799.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
